# 05_lmu_dms_tuning


In [ ]:
#default_exp tune.lmu_dms

In [ ]:
#@title Environment Setup
try:
    # See if we are running on google.colab
    from google.colab import files
    IN_COLAB = True
    !pip install --upgrade nengo nengo-gui nbdev git+https://github.com/neuromorphs/grill-srnn-pfc.git
    !jupyter serverextension enable nengo_gui.jupyter
    # TODO: kaggle creds for downloading data 
    
except ModuleNotFoundError:
    IN_COLAB = False

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#export
from sklearn import metrics
import numpy as np
import nengo
import nengo.utils.least_squares_solvers as lss
import nni
# from srnn_pfc.dms import DMSTask
from srnn_pfc.lmu import make_lmu_dms  # LDN, make_ldn_B_A, 


In [ ]:
#export
def run_trial(params):
    # Create model for training
    train_model, train_probes = make_lmu_dms(
                                    theta=params['theta'],
                                    q=params['q'],
                                    n_neurons=params['n_neurons'],
                                    seed=params['ens_seed'],
                                    out_transform=None,
                                    n_trials_per_cond=params['train_trials_per_cond'],
                                    trial_seed=params['train_trials_seed'],
                                    tau=params['tau'])
    # Generate training data
    train_sim = nengo.Simulator(train_model)
    train_sim.run(6 * 16 * params['train_trials_per_cond'])  # 6 seconds per trial, 16 conditions
    train_sim.close()

    # Solve for weights
    filt = nengo.synapses.Lowpass(0.01)
    A = filt.filt(train_sim.data[train_probes['ensemble']])
    Y = filt.filt(train_sim.data[train_probes['ideal']])
    # TODO: Maybe we only need to slice A and Y around the reporting interval?
    # b_slice = Y != 0
    # D, info = nengo.solvers.LstsqL2(solver=lss.LSMRScipy())(A[b_slice], Y[b_slice])
    #  then we could use a separate gating function
    D, info = nengo.solvers.LstsqL2(solver=lss.LSMRScipy())(A, Y)

    # Create a new model with the learned weights
    test_model, test_probes = make_lmu_dms(
                                  theta=params['theta'],
                                  q=params['q'],
                                  n_neurons=params['n_neurons'],
                                  seed=params['ens_seed'],
                                  out_transform=D.T,
                                  n_trials_per_cond=params['test_trials_per_cond'],
                                  trial_seed=params['test_trials_seed'],
                                  tau=params['tau'])

    # Run test simulation - break after N trials to report cumulative accuracy
    def get_labels_from_sim(sim, n_trials):
        samps = 6 * 1000 * n_trials
        Y = sim.data[test_probes['ideal']][-samps:]
        A = sim.data[test_probes['output']][-samps:]
        b_slice = Y != 0
        label = Y[b_slice].reshape(n_trials, -1)[:, 0]
        score = np.mean(A[b_slice].reshape(n_trials, -1), axis=-1)
        return label, score

    total_test_trials = params['test_trials_per_cond'] * 8 * 2
    test_sim = nengo.Simulator(test_model)

    test_ix = min(20, total_test_trials)
    test_sim.run(6 * test_ix)
    labels, scores = get_labels_from_sim(test_sim, test_ix)

    trial_step = 10
    while test_ix < total_test_trials:
        test_sim.run(6 * trial_step)
        label, score = get_labels_from_sim(test_sim, trial_step)
        labels.append(label)
        scores.append(score)
        fpr, tpr, thresholds = metrics.roc_curve(labels, scores)
        test_auc = metrics.auc(fpr, tpr)
        test_ix += trial_step
    
    test_sim.close()
    nni.report_final_result(test_auc)


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

In [ ]:
#export
try:
    from nbdev.imports import IN_NOTEBOOK
except ModuleNotFoundError:
    IN_NOTEBOOK = False

    
if __name__ == "__main__" and not IN_NOTEBOOK:
    params = nni.get_next_parameter()
    run_trial(params)
